In [2]:
# Imports

import pandas as pd
import qgrid
from sklearn.linear_model import LinearRegression
import os

from tqdm.notebook import tqdm
from highcharts import Highstock

In [3]:
# Calculation functions

def performance(series):
    X = series.index.values.reshape(-1,1)
    y = series.values.reshape(-1,1)

    regressor = LinearRegression()  
    regressor.fit(X, y)
    
    perf = regressor.coef_[0][0]
    yearly = ((perf * 250) / series.iloc[-1]) *100
    return yearly

In [4]:
# Get instruments list

path = '/Users/martin/Google Drive/data/screener/'
files = os.listdir(path)
dfi = pd.DataFrame()
dfi['Instrument'] = files[:]
# Todo: Get instruments from databse

In [5]:
# Add calculations to instruments

for inst in tqdm(dfi['Instrument']):
    df = pd.read_csv(path + inst)
    if len(df) == 0:
        continue
    dfi.loc[dfi['Instrument']==inst, 'Perf_3m'] = performance(df.iloc[-66:]['close'])
    dfi.loc[dfi['Instrument']==inst, 'Perf_1y'] = performance(df.iloc[-250:]['close'])
    dfi.loc[dfi['Instrument']==inst, 'Perf_2y'] = performance(df.iloc[-500:]['close'])

In [6]:
# Add calculations to instruments, multiprocessing

from multiprocessing import Pool

def add_calc(inst):
    df = pd.read_csv(path + inst)
    if len(df) == 0:
        return
    dfi.loc[dfi['Instrument']==inst, 'Perf_3m'] = performance(df.iloc[-66:]['close'])
    dfi.loc[dfi['Instrument']==inst, 'Perf_1y'] = performance(df.iloc[-250:]['close'])
    dfi.loc[dfi['Instrument']==inst, 'Perf_2y'] = performance(df.iloc[-500:]['close'])
    
instrument_list = dfi['Instrument'][:]
with Pool(8) as p:
    p.map(add_calc, instrument_list)

In [7]:
# Build QGrid widget

qgw = qgrid.show_grid(dfi, grid_options={'maxVisibleRows': 30})

In [8]:
# QGrid handlers

def handle_selection_changed(event, widget):
    line_no = event['new'][0]
    build_chart(line_no)
    
qgw.on('selection_changed', handle_selection_changed)


def handle_data_changed(event, widget):
    global dfi
    dfi = qgw.get_changed_df()

qgw.on('filter_changed', handle_data_changed)
qgw.on('sort_changed', handle_data_changed)

In [9]:
# Chart builder

def build_chart(line_no):
    filename = dfi.iloc[line_no,]['Instrument']
    filepath = '/Users/martin/Google Drive/data/screener/'
    df = pd.read_csv(filepath + filename)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
    df['date'] = (df['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')

    chart = Highstock()
    data = df[['date','open','high','low','close']].values.tolist()
    chart.add_data_set(data, series_type='ohlc', name=filename.replace('.csv', ''))
    # todo: add moving average

    options = {
        'chart': {
            'animation': False,
            'height': 800,
            'marginLeft': 50
        },
        'rangeSelector': {
            'selected': 5,
            'buttons': [{
                'type': 'month',
                'count': 1,
                'text': '1m'
            }, {
                'type': 'month',
                'count': 3,
                'text': '3m'
            }, {
                'type': 'month',
                'count': 6,
                'text': '6m'
            }, {
                'type': 'ytd',
                'text': 'YTD'
            }, {
                'type': 'year',
                'count': 1,
                'text': '1y'
            }, {
                'type': 'year',
                'count': 2,
                'text': '2y'
            }, {
                'type': 'all',
                'text': 'All'
            }]
        },
        'navigator': {
            'height': 100
        },
        'title': {
            'text': filename.replace('.csv', '')
        },
        'plotOptions': {
            'series': {
                'animation': False
            }
        },
        'yAxis': {
            'type': 'logarithmic'
        }
    }
    chart.set_dict_options(options)
    chart.save_file('./website/index')

In [11]:
##### Show QGrid widget

qgw

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…